In [14]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from models.utils import DataLoaders, CustomDataset, train_step, eval_step
from models.model import CLIPModel

# Load the dataset from the single folder
dataset = datasets.ImageFolder(root='train')

# Split dataset into 80% train and 20% test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_images, train_labels = zip(*train_dataset)
test_images, test_labels = zip(*test_dataset)
train_dataset = CustomDataset(train_images, train_labels)
test_dataset = CustomDataset(test_images, test_labels)

In [13]:
dl = DataLoaders(train_dataset, test_dataset,'clip',64,True,'texture')
train_loader, test_loader = dl.get_loaders()

In [15]:
# Train and then Evaluate (Three Different Train and Evaluation Loops)
from tqdm import tqdm
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(3)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype, data="texture")
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="texture")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\PCF\.conda\envs\SProj\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\PCF\.conda\envs\SProj\Lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(
100%|██████████| 3/3 [00:09<00:00,  3.33s/it]

Final Train Accuracy: 0.93
Final Test Accuracy: 0.9875
Final Train Loss: 0.7347656716126949
Final Test Loss: 0.09201946606238683
